In [ ]:
import nbaScraper as ns
import actNetScraper as ans
import nbApi as nbapi
from datetime import datetime, timedelta

leagues = ['nba', 'nhl']   # None or list ['nba', 'nhl', 'nfl', 'mlb', 'wnba'] # NONE looks for all sports
specified =  []  ####specific prop, *****only works with a single league in leagues

# day adjustment from today (date of running script), negative = dates into the past
dayJump = 0 
# date can be a list of dates if multiple need scraping 'YYYY-MM-DD'
# default is to only pull today or today + dayJump
dates = [(datetime.today() + timedelta(days=dayJump)).strftime('%Y-%m-%d')]
#dates = ['2024-01-05']#, '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11']

database_export = True  # add all scrapes to database
store_locally = True    # add all scrapes to class variables
season_int = 2025 # int will be the final year of the schedule season
season_str = '2024-25'
season_type = 'Regular+Season' # ['Regular+Season', 'PlayIn', 'Playoffs']
per_mode = 'Totals' #['Totals', 'PerGame']

# nba website and basketball referenece scrapers
scraper = ns.scraper(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    database_export = database_export, 
    store_locally = store_locally,
    pymysql_conn_str =  None
)
# assigns the date of the last time code executed as today
today = scraper.meta_data['today_dt']

# looks up the actual date for the last regular season game date. this will be used to grab the data for the completed games on the date
nbaApi = nbapi.nbaApi(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    database_export = database_export, 
    store_locally = store_locally,
    pymysql_conn_str =  None
)
nbaApi.get_last_game_date(season = season_str)
run_date = nbaApi.last_game_date
dateRange = [
    run_date, run_date
]

prop_scraper = ans.actNetScraper(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    dates = dates,
    leagues = leagues,
    database_export = database_export, 
    store_locally = store_locally,
    config_path = '..\\..\\..\\..\\Notes-General\\config.txt',
    second_run = False
)

# turn to False if issues loading new players
#prop_scraper.update_players = False

# update the league list to only ones with games today
#leagues = prop_scraper.check_for_league_games(date_check = None, update_class_leagues_var = True)
print('scraping for', prop_scraper.leagues, 'on', prop_scraper.dates)

Last day with games played: 2025-03-14
scraping for ['nhl'] on ['2025-03-15']


In [10]:
print(today, 'run date...\n')
prop_scraper.scrape(
    sleep_secs = 3, 
    specific_props = specified,
    leagues_override = prop_scraper.leagues,
    an_state_code = 'BC'
)

prop_scraper.processScrapes(
    remove_dups = True,
    specific_props = specified
)

if prop_scraper.scrape_error_flag:
    print(prop_scraper.scrape_errors)
    #prop_scraper.tryMissingProps()


2025-03-15 run date...

scraping nhl ...
original rows:  (195, 20)
after dups removed:  (195, 20)
[]
nhl odds data loaded...
prop          gs  pts
propId count  20  175


In [ ]:
print(today, 'run date...\n')
#teams = ['GSW','DEN','POR','SAC','TOR','DAL','PHO','CHI','LAL','HOU','MIA','MEM','DET','MIL','NOP','MIN','CLE','OKC','LAC','BRK','SAS','NYK','WAS','CHO','UTA','IND','BOS','PHI','ATL','ORL']
scraper.get_bref_pos_estimates(
        base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp', 
        today_date = today,
        season = season_int,
        database_table = 'brefmisc',
        team_overrides = None
)
### CAN DELETE AFTER CONFIRMING ERROR RETRIES IN FUNCTION WORK AS EXPECTED
#missing_teams = scraper.scrape_errors['brefmisc']['url']
#if len(missing_teams):
#        print('missing:', missing_teams)
#        for i in missing_teams:
#                scraper.get_bref_pos_estimates(
#                        base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp', 
#                        today_date = today,
#                        season = season_int,
#                        database_table = 'brefmisc',
#                        team_overrides = [i[0]]
#                )
    

2025-03-15 run date...

bref player pos estimates scraped...
missing: [['MIA', 2025], ['IND', 2025], ['PHI', 2025], ['ORL', 2025]]
bref player pos estimates scraped...
bref player pos estimates scraped...
bref player pos estimates scraped...
bref player pos estimates scraped...


In [ ]:
# API hits
print(today, 'run date...\n')
# --- TEAM --- #
nbaApi.request_nba_team_playtype_data(
            season = season_str, #'YYYY-YY' 
            play_type = [
                'Isolation', 'Transition','PRBallHandler','PRRollman', 'Postup', 'Spotup', 
                'Handoff', 'Cut', 'OffScreen', 'OffRebound', 'Misc'
            ], 
            lid='00',
            per_mode = 'PerGame', #['Totals', 'PerGame'] 
            season_type = 'Regular+Season', #['Regular+Season', 'PlayIn', 'Playoffs']
            sob = ['offensive', 'defensive'],
            sleep_time = 2,
            database_table = 'statsteamplaytypes'
        )

nbaApi.request_nba_team_shotzone_data(
    season = season_str, #'YYYY-YY'
    start_date = run_date, #'MM/DD/YYY'
    end_date = run_date, 
    per_mode = per_mode, #['Totals', 'PerGame'] 
    season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
    distance_type = 'By+Zone', # ['8ft+Range', '5ft+Range','By+Zone']
    sob = ['Base', 'Opponent'], #['Base', 'Opponent']  base= teams offense, opponent = teams defense
    database_table = 'statsteamshotzones'     
)

# PER GAME
nbaApi.request_nba_team_stats(
    season = season_str, #'YYYY-YY'
    start_date = run_date, #'MM/DD/YYY'
    end_date = run_date, 
    per_mode = 'PerGame', #['Totals', 'PerGame'] 
    season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
    measure_type = ['Base', 'Advanced', 'Opponent'], #['Base', 'Advanced', 'Opponent']  
    database_table = 'statsteam'
)
# TOTALS
nbaApi.request_nba_team_stats(
    season = season_str, #'YYYY-YY'
    start_date = run_date, #'MM/DD/YYY'
    end_date = run_date, 
    per_mode = 'Totals', #['Totals', 'PerGame'] 
    season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
    measure_type = ['Base', 'Advanced', 'Opponent'], #['Base', 'Advanced', 'Opponent']  
    database_table = 'statsteamtotals'
)

# --- PLAYER --- #
nbaApi.request_nba_player_playtype_data(
            season = season_str, #'YYYY-YY' 
            play_type = [
                'Isolation', 'Transition','PRBallHandler','PRRollman', 'Postup', 'Spotup', 
                'Handoff', 'Cut', 'OffScreen', 'OffRebound', 'Misc'
            ], 
            lid='00',
            per_mode = 'PerGame', #['Totals', 'PerGame']
            season_type = 'Regular+Season', #['Regular+Season', 'PlayIn', 'Playoffs']
            sob = ['offensive'], #'offensive','defensive'],
            sleep_time = 2,
            database_table = 'statsplayerplaytypes'
)

nbaApi.request_nba_player_shotzone_data(
            season = season_str, #'YYYY-YY'
            start_date = run_date, #'MM/DD/YYY'
            end_date = run_date, 
            per_mode = per_mode, #['Totals', 'PerGame'] 
            season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
            distance_type = 'By+Zone', # ['8ft+Range', '5ft+Range','By+Zone'],
            sob = ['Base'], #['Base', 'Opponent']  base= teams offense, opponent = teams defense
            database_table = 'statsplayershotzone'        
)

# Passing
nbaApi.request_nba_player_tracking(
        season = season_str, #'YYYY-YY'
        start_date =run_date, #'MM/DD/YYY'
        end_date = run_date, 
        per_mode = per_mode, #['Totals', 'PerGame'] 
        season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
        measure_type = 'Passing', #['Passing','Rebounding','Drives','Possessions','Efficiency','PostTouch', 'ElbowTouch', 'PaintTouch']  
        database_table = 'statsplayerpassing'      
)

# Rebounding
nbaApi.request_nba_player_tracking(
        season = season_str, #'YYYY-YY'
        start_date =run_date, #'MM/DD/YYY'
        end_date = run_date, 
        per_mode = per_mode, #['Totals', 'PerGame'] 
        season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
        measure_type = 'Rebounding', #['Passing','Rebounding','Drives','Possessions','Efficiency']  
        database_table = 'statsplayerrebounding'      
)

# add these measure types - 
#### 'Drives', 'Possessions', 'Efficiency' 
# maybe add these - 'PostTouch', 'ElbowTouch', 'PaintTouch'
#nbaApi.request_nba_player_tracking()

In [ ]:
df = nbaApi.
df

,team,gp,w,l,winPct,min,pts,fgm,fga,fgPct,...,oppReb,oppAst,oppTo,oppStl,oppBlk,oppBlka,oppPf,oppPfd,oppPts,oppPlusMinsus
0,Atlanta Hawks,1,0,1,0.0,48.0,37.0,88.0,0.420,9.0,...,40.0,27.0,14.0,14.0,5.0,6.0,15.0,18.0,121.0,23.0
1,Boston Celtics,1,1,0,1.0,48.0,36.0,83.0,0.434,19.0,...,38.0,18.0,18.0,10.0,4.0,4.0,12.0,17.0,91.0,-12.0
2,Charlotte Hornets,1,1,0,1.0,48.0,51.0,85.0,0.600,26.0,...,36.0,27.0,14.0,14.0,5.0,2.0,17.0,17.0,134.0,-11.0
3,Cleveland Cavaliers,1,1,0,1.0,48.0,44.0,88.0,0.500,19.0,...,45.0,25.0,14.0,11.0,3.0,4.0,25.0,17.0,124.0,-9.0
4,Dallas Mavericks,1,0,1,0.0,48.0,39.0,94.0,0.415,8.0,...,67.0,27.0,11.0,5.0,6.0,6.0,13.0,11.0,133.0,37.0
5,Denver Nuggets,1,1,0,1.0,48.0,44.0,85.0,0.518,13.0,...,47.0,32.0,9.0,10.0,2.0,5.0,25.0,15.0,126.0,-5.0
6,Houston Rockets,1,1,0,1.0,48.0,54.0,105.0,0.514,15.0,...,34.0,14.0,8.0,4.0,6.0,6.0,11.0,13.0,96.0,-37.0
7,Indiana Pacers,1,1,0,1.0,48.0,44.0,89.0,0.494,10.0,...,43.0,13.0,16.0,7.0,9.0,4.0,19.0,17.0,100.0,-12.0
8,LA Clippers,1,1,0,1.0,48.0,47.0,86.0,0.547,12.0,...,44.0,25.0,18.0,8.0,6.0,5.0,18.0,15.0,98.0,-23.0
9,Los Angeles Lakers,1,0,1,0.0,48.0,49.0,105.0,0.467,15.0,...,39.0,30.0,12.0,7.0,5.0,2.0,15.0,25.0,131.0,5.0


In [ ]:
#prop_scraper.tryMissingProps()
# this didn't work as expected, 
#### added SOG to the nba scrape_errors
#### added new key-value pair for the missing prop:  'missing_dates': ['sog', '2025-01-02', 0]

In [ ]:
prop_scraper.scrape_errors

# loading misscraped passing and rebound date 

In [7]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamefinder

In [93]:
cstring = scraper.pymysql_conn_str
# qery db to get dates of messed up dates
df = pd.read_sql_query(
    #sql = """SELECT date, COUNT(PLAYER_NAME) n FROM statsplayershotzones GROUP BY date HAVING date > '2024-10-20' and n = 50 ORDER BY date;""",
    sql = "SELECT DISTINCT date FROM statsteamtotals WHERE date > '2023-10-20' AND date < '2024-07-01';",
    con = cstring
)
#dates = df[df['n'] == 50]['date'].tolist()
dates = df['date'].tolist()


In [94]:
db = [pd.to_datetime(i).date() for i in dates]
actual = dates_reg_2324 + dates_post_2324
np.setdiff1d(actual, db)

array([], dtype=object)

In [99]:
 # Fetch all games for the current season
seasons = '2024-25'
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=seasons, league_id_nullable='00')
games2425 = gamefinder.get_data_frames()[0]
games2425.loc[:,'GAME_DATE'] = pd.to_datetime(games2425['GAME_DATE'])
games2425 = games2425[games2425['SEASON_ID'].astype(str).str.startswith(('2', '4', '5'))]
dates_reg_2425 = games2425[games2425['SEASON_ID'].astype(str).str.startswith(('2'))]['GAME_DATE'].unique().tolist()
dates_reg_2425 = [i.date() for i in dates_reg_2425]
dates_reg_2425.sort()
#dates_post_2425 = games2425[games2425['SEASON_ID'].astype(str).str.startswith(('4', '5'))]['GAME_DATE'].unique().tolist()
#dates_post_2425 = [i.date() for i in dates_post_2324]
#dates_post_2425.sort()


In [ ]:
### HISTORICAL LOADERS
####
for i in []:
    print(i)
    d = pd.to_datetime(i)
    dateRange = [d, d]
    scraper.get_nba_team_shotzone_data(            
        base_url = 'https://www.nba.com/stats/teams/{sideOfBall}?DistanceRange=By+Zone&SeasonType={type}&DateFrom={startDate}&DateTo={endDate}&PerMode={perMode}&Season={season}', 
        sides = {'offensive':'shooting', 'defensive':'opponent-shooting'},
        season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        perMode = 'Totals', #[Totals, PerMode]
        database_table = 'statsteamshotzones',
        dateRange = dateRange,
        season = '2023-24'
    )


for i in []:
    print(i)
    d = pd.to_datetime(i)
    dateRange = [d, d]
    scraper.get_nba_player_shotzone_data(
        #base_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}&DateTo={endDate}', 
        base_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&SeasonType={type}&DateFrom={startDate}&DateTo={endDate}&PerMode=Totals&Season=2023-24',  
        season_type = 'PlayIn',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        lastNgames = 0,
        database_table = 'statsplayershotzones',
        dateRange = dateRange
    )


for i in []:
    print(i)
    d = pd.to_datetime(i)
    scraper.get_nba_player_passing_data(
        base_url = 'https://www.nba.com/stats/players/passing?DateFrom={d1}&DateTo={d2}&PerMode=Totals&SeasonType={type}&Season=2023-24', 
        run_date = d,
        season_type = 'Playoffs',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        database_table = 'statsplayerpassing'
    )

#
for i in []:
    print(i)
    d = pd.to_datetime(i)
    scraper.get_nba_player_rebounding_data(
        base_url = 'https://www.nba.com/stats/players/rebounding?DateFrom={d1}&DateTo={d2}&PerMode=Totals&SeasonType={type}&Season=2023-24', 
        run_date = d,
        season_type = 'Playoffs',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        database_table = 'statsplayerrebounding'
    )

p = 'Regular+Season'
for i in dates_reg_2425:
    print(i)
    d = pd.to_datetime(i)
    dateRange = [d, d]  
    # per game
    scraper.get_nba_team_stats(
        base_url = 'https://www.nba.com/stats/teams/{stats}?SeasonType={seasonType}&DateFrom={startDate}&DateTo={endDate}&PerMode={perMode}&Season={season}',
        stats = ['traditional', 'advanced', 'opponent'],
        season_type = p,  # ['Regular+Season', 'PlayIn', 'Playoffs']
        perMode = 'PerGame',  # [Totals, PerGame]
        database_table = 'statsteam',
        dateRange = dateRange,
        season = '2024-25'
    )
    # totals
    scraper.get_nba_team_stats(
        base_url = 'https://www.nba.com/stats/teams/{stats}?SeasonType={seasonType}&DateFrom={startDate}&DateTo={endDate}&PerMode={perMode}&Season={season}',
        stats = ['traditional', 'advanced', 'opponent'],
        season_type = p,  # ['Regular+Season', 'PlayIn', 'Playoffs']
        perMode = 'Totals',  # [Totals, PerGame]
        database_table = 'statsteamtotals',
        dateRange = dateRange,
        season = '2024-25'
    )

# scratch

In [29]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time 

In [30]:
service = Service('..\\..\\browser\\geckodriver.exe')
driver = webdriver.Firefox(service=service)
url = 'https://www.nba.com/stats/players/rebounding?DateFrom=10%2F24%2F2024&DateTo=10%2F24%2F2024'
driver.get(url)

In [31]:
driver.close()

In [76]:
service = Service('..\\..\\browser\\geckodriver.exe')
driver = webdriver.Firefox(service=service)
# loop through each team webpage to gather data        
base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp'

season = 2025

#all_team_data = []
url_errors = []

# basketball ref team url abbrevs
bref_team_abbr = [
    'GSW','DEN','POR', 'SAC','TOR',
    'DAL','PHO','CHI','LAL','HOU',
    'MIA','MEM','DET','MIL','NOP','MIN',
    'CLE','OKC','LAC','BRK','SAS','NYK','WAS','CHO',
    'UTA','IND','BOS','PHI','ATL','ORL'
]

# col names in the database
bref_cols = [
    'player', 'age', 'pos', 'gp', 'gs', 'mp', 'PG', 'SG', 'SF',
    'PF', 'C', 'onCourtPlusMinusPer100', 'onOffPlusMinusPer100',
    'badPass', 'lostBall', 'shootFoulCommitted', 'offFoulCommitted',
    'shootFoulDrawn', 'offFoulDrawn', 'ptsGenFromAst', 'andOnes', 'shotsBlk', 'awards',
    'date', 'team'
]

all_team_data = pd.DataFrame(columns=bref_cols)

# loop through each team webpage to gather data        
for i in bref_team_abbr:
    time.sleep(2)
    try:
        url = base_url.format(team = i, season = str(season))
        driver.get(url)
        time.sleep(2)
        
        #try:
        #    popup = driver.find_element(By.ID, 'modal-close')
            # Wait for the popup to be present and visible
            #popup = WebDriverWait(driver, 3).until(
            #    EC.visibility_of_element_located(By.ID, 'modal-close')
            #)
        #    popup.click()
        #except: continue

        table_id = 'pbp_stats'
        table = None  # Placeholder for the table element
        scroll_attempts = 30  # Number of scrolling attempts
        scroll_step = 500  # Pixels to scroll down on each attempt
  
        # this was removed out of the loop below. loop stopped working and selenium is now able to find it without scrolling
        table = driver.find_element(By.ID, table_id)
        driver.execute_script("arguments[0].scrollIntoView();", table)

        table_html = table.get_attribute('outerHTML')  # Get table HTML
        df = pd.read_html(table_html)[0]  # Convert to DataFrame
        df = df.iloc[:,1:].reset_index(drop=True)
        df.loc[:,'date'] = '2025'
        df.loc[:,'team'] = i
        df.columns = bref_cols

        all_team_data = pd.concat([all_team_data, df])
    
    except:
        print('error', i)
        continue
    

driver.close()
#bref_pos_estimates = pd.DataFrame(all_team_data, columns = bref_cols)

C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_team_data = pd.concat([all_team_data, df])


error DEN


C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame


error TORDAL


C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame


error CHI
error LAL
error HOU
error MIA
error MEM
error DET
error MIL
error NOP
error MIN
error CLE
error OKC
error LAC


KeyboardInterrupt: 

In [75]:
for i in all_team_data.team.unique():
    print(i)

GSW
DEN
POR
SAC
PHO
CHI
LAL
HOU
MIA
MEM
DET
NOP
MIN
CLE
OKC
LAC
BRK
SAS
NYK
CHO
UTA
IND
BOS
PHI
ATL
ORL


In [41]:
driver.close()

In [29]:
popup = driver.find_element(By.ID, 'modal-close')
popup.click()

In [31]:
# this was removed out of the loop below. loop stopped working and selenium is now able to find it without scrolling
table = driver.find_element(By.ID, table_id)

driver.execute_script("arguments[0].scrollIntoView();", table)

In [32]:
table_html = table.get_attribute('outerHTML')  # Get table HTML
df = pd.read_html(table_html)[0]  # Convert to DataFrame
df = df.iloc[:,1:].reset_index(drop=True)

C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\4285168731.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame


In [34]:
table

<selenium.webdriver.remote.webelement.WebElement (session="640c6a3f-f302-445d-a3ed-fa522d4d8c12", element="bea42cb3-e84f-4cba-b374-3502da78fdf0")>